In [1]:
! pip install anthropic pyttsx4 pillow
CLAUDE_API_KEY=""

Defaulting to user installation because normal site-packages is not writeable


Installing the Libraries

In [ ]:
# import cv2
# import pytesseract as py

# import nltk
# nltk.download('punkt')
# nltk.download('stopwords')
# from nltk.cluster.util import cosine_distance
# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize
# from nltk.tokenize import sent_tokenize
# from nltk.probability import FreqDist
# from heapq import nlargest
# import networkx as nx

# import numpy as np
# import os

In [5]:
"""
Summarizes the contents of a file and displays the summary in a text box. Additionally, the summary is read aloud using a text-to-speech engine.

Args:
    file_path (str): The path to the file to be summarized.

Returns:
    None
"""
import base64
import pyttsx4
import tkinter as tk
from anthropic import Anthropic
from PIL import Image, ImageTk
from threading import Thread
from tkinter import filedialog

class App(tk.Tk):
    def __init__(self):
        super().__init__()
        
        self.engine = pyttsx4.init()
        self.engine.setProperty('rate', 150)
        self.engine.setProperty('voice', self.engine.getProperty('voices')[1].id)

        self.title("News Summarizer")
        self.geometry('1200x600')

        upload_button = tk.Button(self, text="Upload Image here", command=self.open_image)
        upload_button.pack(pady=10)

        self.image_label = tk.Label(self, height=500, width=500)
        self.image_label.pack(side=tk.LEFT, padx=10)

        self.sbox = tk.Text(self, height=30, width=50)
        self.sbox.config(state="disabled", bg='#dddddd')
        self.sbox.pack()

        btn = tk.Button(self, text= "Summarize", command=self.get_summary)
        btn.pack(pady=10)

    def open_image(self):
        self.file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg"), ("All files", "*.*")])
        image = Image.open(self.file_path)
        image.thumbnail((500, 500))
        self.photo = ImageTk.PhotoImage(image)
        self.image_label.configure(image=self.photo)
        self.image_label.image = self.photo

    def get_summary(self):
        self.sbox.config(state='normal')
        self.sbox.delete('1.0', 'end')
        Thread(target=self.get_claude_summary).start()
    
    def update_summary_in_main_thread(self, text):
        self.sbox.insert(tk.END, text)
        self.sbox.see(tk.END)
        
    def get_claude_summary(self):
        finaltext = ""
        for event in self.get_summary_stream():
            if event.type == "content_block_start":
                text = event.content_block.text
            elif event.type == "content_block_delta":
                text = event.delta.text
            else:
                continue
            self.after(1, self.update_summary_in_main_thread, text)
            finaltext += text
        self.engine.say(finaltext)
        Thread(target=self.engine.runAndWait).start()

    def get_summary_stream(self):
        """
        Summarizes the text content of an image using the Claude AI model.
        
        Args:
            filename (str): The file path of the image to be summarized.
        
        Returns:
            str: The summary of the text content in the image.
        """
        api_key = CLAUDE_API_KEY
        image_file = open(self.file_path, "rb")
        encoded_string = base64.b64encode(image_file.read()).decode('utf-8')
        image_file.close()
        client = Anthropic(api_key=api_key)
        stream = client.messages.create(
            model="claude-3-sonnet-20240229",
            max_tokens=1024,
            stream=True,
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "image",
                            "source": {
                                    "type": "base64",
                                    "media_type": "image/jpeg",
                                    "data": encoded_string,
                                },
                            },
                            {
                                "type": "text",
                                "text": "Summarise the text in this image."
                            }
                    ]
                }
            ]
        )
        return stream

app = App()
app.mainloop()


Exception in thread Thread-10 (get_claude_summary):
Traceback (most recent call last):
  File "c:\Program Files\Python310\lib\threading.py", line 1009, in _bootstrap_inner
    self.run()
  File "C:\Users\DELL\AppData\Roaming\Python\Python310\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Program Files\Python310\lib\threading.py", line 946, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\DELL\AppData\Local\Temp\ipykernel_6280\3154696828.py", line 61, in get_claude_summary
  File "C:\Users\DELL\AppData\Local\Temp\ipykernel_6280\3154696828.py", line 88, in get_summary_stream
  File "C:\Users\DELL\AppData\Roaming\Python\Python310\site-packages\anthropic\_utils\_utils.py", line 277, in wrapper
    return func(*args, **kwargs)
  File "C:\Users\DELL\AppData\Roaming\Python\Python310\site-packages\anthropic\resources\messages.py", line 681, in create
    return self._post(
  File "C:\Users\DELL\AppData\Roaming\Pyth

Summarizing algorithm

In [12]:
# !pip install anthropic
def summariser(filename):
    api_key=""
    from anthropic import Anthropic
    # read image in base 64 encoded format
    import base64
    image_file = open(filename, "rb")
    encoded_string = base64.b64encode(image_file.read()).decode('utf-8')
    image_file.close()
    client = Anthropic(api_key=api_key)
    message = client.messages.create(
    model="claude-3-sonnet-20240229",
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "source": {
                            "type": "base64",
                            "media_type": "image/jpeg",
                            "data": encoded_string,
                        },
                    },
                    {
                        "type": "text",
                        "text": "Summarise the text in this image."
                    }
            ]
        }
    ]
    )
    print(message.content[0].text)
    return message.content[0].text

In [13]:
"""
Summarizes the contents of a file and displays the summary in a text box. Additionally, the summary is read aloud using a text-to-speech engine.

Args:
    file_path (str): The path to the file to be summarized.

Returns:
    None
"""
import base64
import pyttsx4
import tkinter as tk
from anthropic import Anthropic
from threading import Thread
from tkinter import filedialog

class App(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("News Summarizer")
        self.geometry('1200x600')

        upload_button = tk.Button(self, text="Upload Image here", command=self.open_image)
        upload_button.pack()

        title = tk.Label(self, text="Title")
        title.pack()

        tbox= tk.Text(self, height=1, width=140)
        tbox.config(state='disabled', bg='#dddddd')
        tbox.pack()
        
        summary = tk.Label(self, text="Summary")
        summary.pack()

        self.sbox = tk.Text(self, height=20, width=140)
        self.sbox.config(state="disabled", bg='#dddddd')
        self.sbox.pack()

        btn = tk.Button(self, text= "Summarize", command=self.get_summary)
        btn.pack()

    def open_image(self):
        self.file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg"), ("All files", "*.*")])

    def get_summary(self):
        self.sbox.config(state='normal')
        self.sbox.delete('1.0', 'end')
        Thread(target=self.get_claude_summary).start()
    
    def update_summary_in_main_thread(self, text):
        self.sbox.insert(tk.END, text)
        self.sbox.see(tk.END)
        
    def get_claude_summary(self):
        finaltext = ""
        num_lines = 1
        engine = pyttsx4.init()
        for event in self.get_summary_stream():
            if event.type == "content_block_start":
                text = event.content_block.text
            elif event.type == "content_block_delta":
                text = event.delta.text
            else:
                continue
            self.after(1, self.update_summary_in_main_thread, text)
            finaltext += text
            # lines = /
            if len() > num_lines:
                num_lines = finaltext.split(". ") + 1

        engine.say(finaltext)
        engine.runAndWait()

    def get_summary_stream(self):
        """
        Summarizes the text content of an image using the Claude AI model.
        
        Args:
            filename (str): The file path of the image to be summarized.
        
        Returns:
            str: The summary of the text content in the image.
        """
        api_key = CLAUDE_API_KEY
        image_file = open(self.file_path, "rb")
        encoded_string = base64.b64encode(image_file.read()).decode('utf-8')
        image_file.close()
        client = Anthropic(api_key=api_key)
        stream = client.messages.create(
            model="claude-3-sonnet-20240229",
            max_tokens=1024,
            stream=True,
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "image",
                            "source": {
                                    "type": "base64",
                                    "media_type": "image/jpeg",
                                    "data": encoded_string,
                                },
                            },
                            {
                                "type": "text",
                                "text": "Summarise the text in this image."
                            }
                    ]
                }
            ]
        )
        return stream

app = App()
app.mainloop()
